In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import f1_score

In [30]:
def find_fibra_quota(x):
    if not x.find("FIBRA")==-1:
        ix = x.find("FIBRA")+6
        iy = x[ix:].find(" ")
        iz = x[(ix+iy+1):].find(" ")
        whatwehave = x[ix: (ix+iy+iz+1)]
        if whatwehave.find("GB")!= -1:
            return float(whatwehave[:whatwehave.find(" ")])*1000
        else:
            try:
                return float(whatwehave[:whatwehave.find(" ")])
            except ValueError as e:
                print(x,end=" :")
                print(e)
                print("tried to convert: "+whatwehave)
                print("returning this: "+str(float(whatwehave)*1000))
                return float(whatwehave)*1000
    return 100

### Read Data

In [93]:
main = pd.read_csv("./data/train.csv")
main_test = pd.read_csv("./data/test.csv")
main = pd.concat([main,main_test],axis=0)

In [136]:
main = pd.read_csv("./data/train_feb.csv")

### Define Columns

In [137]:
target = "Churn.30"
# target = "TARGET"
relevant_columns = main.columns.tolist()
relevant_columns.remove(target)

### Exploring the columns

In [63]:
for col in relevant_columns:
    print("="*5,end=" ")
    print(col,end=" ")
    print("="*5)
    print(main[col].value_counts(dropna=False))

===== id =====
2047     1
38232    1
9550     1
15693    1
13644    1
3403     1
1354     1
7497     1
5448     1
27975    1
25926    1
32069    1
30020    1
19779    1
17730    1
23873    1
21824    1
44351    1
42302    1
48445    1
46396    1
36155    1
34106    1
11599    1
40281    1
11503    1
34138    1
11631    1
9582     1
15725    1
        ..
41601    1
43648    1
21119    1
23166    1
17021    1
25241    1
31386    1
29339    1
37543    1
4791     1
6838     1
693      1
2740     1
12979    1
15026    1
8881     1
10928    1
49837    1
39590    1
19100    1
33445    1
35492    1
45731    1
47778    1
41633    1
43680    1
21151    1
23198    1
17053    1
0        1
Name: id, Length: 50318, dtype: int64
===== ultimo_ivr_departamento =====
ATC                 47965
SOPORTE A VENTAS     1363
VENTAS                759
COBROS                110
CANCELACIONES          60
Missing                60
ATC NEGOCIO             1
Name: ultimo_ivr_departamento, dtype: int64
===== ultima_c

0     48193
1      1509
2       397
3       122
4        43
5        27
6        13
9         4
8         4
11        3
7         3
Name: times_retention, dtype: int64
===== zip_code =====
41006.0    188
11100.0    159
6800.0     158
45600.0    146
11130.0    143
11500.0    143
28041.0    135
28021.0    134
8830.0     126
11300.0    123
41500.0    119
28019.0    118
29640.0    116
29010.0    114
28017.0    114
28053.0    113
29620.0    111
Missing    110
28850.0    109
11540.0    105
28025.0    105
41008.0    104
41710.0    102
28038.0    100
11408.0    100
41015.0     98
28018.0     97
28031.0     96
28032.0     94
28039.0     94
          ... 
49133.0      1
46669.0      1
27547.0      1
19192.0      1
15847.0      1
50650.0      1
46725.0      1
39360.0      1
44370.0      1
33592.0      1
46630.0      1
16670.0      1
49157.0      1
9690.0       1
15689.0      1
46790.0      1
24250.0      1
35350.0      1
4729.0       1
39571.0      1
3749.0       1
4210.0       1
46703.0      1
1

9     18940
4      7905
6      5590
7      5122
1      4243
10     4187
0      3647
8       315
3       180
5       111
2        78
Name: province_clustered_11, dtype: int64
===== port_in_operator_ds_clustered_11 =====
8     23296
0     14844
5      9056
7      1378
10      758
3       622
2       196
4       124
9        30
6        11
1         3
Name: port_in_operator_ds_clustered_11, dtype: int64
===== client_segment_clustered_11 =====
6     23399
8     11643
10     7927
9      5892
3       639
4       604
0       109
7        48
2        31
5        17
1         9
Name: client_segment_clustered_11, dtype: int64
===== total_segundos_espera_transformed =====
1    38828
0    11490
Name: total_segundos_espera_transformed, dtype: int64
===== total_segundos_conversacion_transformed =====
1    44318
0     6000
Name: total_segundos_conversacion_transformed, dtype: int64
===== ranking_transformed =====
1    36579
0    13739
Name: ranking_transformed, dtype: int64
===== tariff_ds_fibra_quot

### Correct type of the columns

In [138]:
main["grupo_acd"] = main["grupo_acd"].astype(str)
main["locality_area"] = main["locality_area"].astype(str)
main["outcome_baja"] = main["outcome_baja"].astype(str)
main["zip_code"] = main["zip_code"].astype(str)
main["times_retention"] = main["times_retention"].astype(str)

### Create New Features

In [139]:
main["client_segment"] = main.grupo_acd.apply(lambda x: x[-3:])
main["bpo"] = main.grupo_acd.apply(lambda x: x[:2])

In [140]:
main["tariff_ds_has_fibra"] = main.tariff_ds.apply(lambda x: x.find("FIBRA")!=-1)
main["tariff_ds_fibra_quota"] = main.tariff_ds.apply(find_fibra_quota)

LA FIBRA 1 GB :could not convert string to float: 
tried to convert: 1
returning this: 1000.0


### Export Feature Dictionary & Final Data

In [141]:
dtypes_ = dict(zip(main.dtypes.index,main.dtypes))
dtypes_.pop(target)
dtypes_.pop('id_agente')
# dtypes_.pop('id')
# dtypes_.pop('TARGET')
# dtypes_.pop('id')

dtype('int64')

In [142]:
dtypes_df = pd.DataFrame.from_dict(dtypes_,orient="index",columns=["dtype"])
dtypes_df.reset_index(inplace=True)
dtypes_df.rename(columns={"index":"feature"},inplace=True)
dtypes_df["dtype2"] = dtypes_df.dtype.apply(lambda x: "cont" if x in [np.dtype("int64"),np.dtype("float64")] else "cat")
dtypes_df["dtype2"] = dtypes_df.apply(lambda x: "cat" if (x.feature.find("_transformed")!=-1 or x.feature.find("_clustered")!=-1) else x.dtype2, axis=1)

In [143]:
dtypes_df["distinct_count"] = dtypes_df.feature.apply(lambda x: main[x].value_counts(dropna=False).shape[0]) 
dtypes_df["na_count"] = dtypes_df.feature.apply(lambda x: main[main[x].isna()].shape[0])
dtypes_df["na_perc"] = dtypes_df.na_count.apply(lambda x: x/main.shape[0])

In [144]:
dtypes_df.to_csv('feature_dictionary_feb.csv')

In [153]:
def fill_nas(main):
    for col in dtypes_df.feature:
        try:
            if col in cat_features:
                main[col].fillna("Missing",inplace=True)
            else:
                main[col] = main[col].apply(lambda x: x if (x!=None) and (x!=np.nan) and (np.isnan(x)!=True) else main[col].mean())
        except:
            print(col+" cannot be found (or something else happened.)")
    return main

In [154]:
main = fill_nas(main)

In [168]:
main.to_pickle('./data/model_data_feb.pkl')